### Reading Qualifying.json file

In [0]:
dbutils.notebook.run("../Mounting_storage", 120)

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
qualify_schema = StructType(fields=[StructField("qualifyId", IntegerType(), False),
                                      StructField("raceId", IntegerType(), True),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("constructorId", IntegerType(), True),
                                      StructField("number", IntegerType(), True),
                                      StructField("position", IntegerType(), True),
                                      StructField("q1", StringType(), True),
                                      StructField("q2", StringType(), True),
                                      StructField("q3", StringType(), True)
                                     ])

In [0]:
df_qualifying = spark.read.option("multiline", True).schema(qualify_schema).json("/mnt/blobstorage/qualifying_split_*.json")

display(df_qualifying)

### Renaming and adding new columns

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
final_df = df_qualifying.withColumnRenamed("qualifyId", "qualify_id")\
                     .withColumnRenamed("driverId", "driver_id") \
                     .withColumnRenamed("raceId", "race_id") \
                     .withColumnRenamed("constructorId", "constructor_id")\
                     .withColumn("ingestion_time", current_timestamp())

display(final_df)

### Write it to Parquet format

In [0]:
final_df.write.mode("overwrite").parquet("/mnt/blobstorage/qualifying")

In [0]:
%fs
ls mnt/blobstorage/qualifying